# Setup
Local Spark Environment (3.3.1)
- 4 Cores, 8 Threats (1.8 Ghz Base Clock)
- 8 GB RAM (shared with OS)
- SSD 

Durchlaufen der Systems der Daten
1. Einlesen der Daten
2. Erstellen eines Lesestroms für die Daten, mit maximal 2 Dateien pro Trigger
3. Durchlaufen der Interferenzpipeline

Pfade:
Memory-bound: Bearbeiten der Daten im Zwischenspeicher (RAM)
CPU-bound: Mit allen Daten im RAM sind wir CPU-Bound

Skalierung: 
Unsere Anwendung ist noch nicht für die Verarbeitung von Big Data skaliert, da sie auf einem einzelnen lokalen System ausgeführt wird (Local Mode). Erst wenn die Menge an Daten, größer als der verfügbare Arbeitsspeicher des lokalen Systems ist, werden die Leistungsprobleme und Out-of-Memory-Fehler auftreten.

Was passiert wenn die Datenmengen größenordnungsmäßig zunehmen? (1x/10x/100x... oder 2x/4x/8x/16x/...)?
- Erhöhte Verarbeitungsleistung: Das Upscaling der Live-Analyse erfordert in der Regel eine höhere Verarbeitungsleistung, da mehr Daten in Echtzeit analysiert werden müssen. Dies führt zu dem Einsatz von mehr oder leistungsfähigerer Hardware. Hierbei ist zu beachten dass eine horizontale Skalierung effizienter ist.
- Erhöhte Speicherkapazität: Das Upscaling der Live-Analyse erfordert auch mehr Speicherkapazität, um die zusätzlichen Daten zu speichern, die analysiert werden. Dies könnte die Verwendung größerer Festplatten oder zusätzlicher Speichersysteme bedeuten.
- Erhöhte Bandbreite: Die Hochskalierung der Live-Feed-Datenanalyse erfordert möglicherweise mehr Bandbreite für die Übertragung der zusätzlichen Daten, die analysiert werden. Dies kann zu einem Problem werden, wenn die Netzwerkinfrastruktur nicht in der Lage ist, die erhöhte Last zu bewältigen.

Was passiert wenn die Anfragen oder Abfragen größenordnungsmäßig zunehmen oder die Latenzbedingungen größenordnungsmäßig abnehmen?
- Erhöhte Verarbeitungszeit: Wenn die Anfragen oder Abfragen größer werden, kann ihre Verarbeitung länger dauern, da mehr Daten zu analysieren sind. Dies kann zu erhöhten Latenzzeiten und einer insgesamt langsameren Leistung führen.
- Erhöhte Ressourcennutzung: Größere Anfragen oder Abfragen erfordern unter Umständen auch mehr Ressourcen, z. B. Verarbeitungsleistung, Arbeitsspeicher und Speicherplatz, um verarbeitet zu werden. Dies kann zu einer erhöhten Ressourcennutzung führen und möglicherweise die Leistung anderer Aufgaben oder Prozesse, die gleichzeitig laufen, beeinträchtigen.
- Geringere Leistung: Wenn die Latenzbedingungen abnehmen, kann es schwieriger werden, Anfragen oder Abfragen in Echtzeit zu verarbeiten, da weniger Zeit für die Verarbeitung der Daten zur Verfügung steht. Dies kann zu einer verringerten Leistung und möglicherweise zu verpassten Analysemöglichkeiten führen.
- Erhöhte Komplexität: Sowohl größere Anfragen oder Abfragen als auch geringere Latenzzeiten können die Datenanalyse komplexer machen, da mehr Daten zu verarbeiten sind und weniger Zeit zur Verfügung steht. Dies kann den Einsatz fortschrittlicherer Algorithmen oder Techniken erforderlich machen, um das größere Datenvolumen zu bewältigen und die Echtzeitverarbeitungsanforderungen zu erfüllen.

Welche Pfade sind einfach/schwieriger Skalierbar?
- Datenerfassung: Einfache Skalierbarkeit, da weitere Server hinzugefügt werden können, um die erhöhte Last zu bewältigen und mehr Daten von der Twitter-API zu sammeln.
- Datenbereinigung: Aktueller Prozess zur Datenbereinigung relativ simpel (entfernen von Sonderzeichen, Kleinschreibung, etc.) somit kann der Pfad gut skalieren.
- Datenumwandlung: Aktuell ist die Datenumwandlung noch simpel, und das System kann gut skalieren. Falls die Datenumwandlung  jedoch komplexer werden sollte (Machine Learning, Erkennen von Text in Bildern/Audio konvertieren in Text, etc.). würden würde das die Skalierung erheblich schwieriger machen.
- Modellinterferenz: Skaliert gut, da nur ein begrenzter Datensatz für das Training genommen werden kann, welcher gelabelt wurde. 
- Datenspeicherung: Dieser Schritt ist relativ einfach zu skalieren, da weitere Server hinzugefügt werden können, um die erhöhte Last zu bewältigen, und die Daten auf die Server verteilt werden können.

Dimensionen Zielsystem Prototyp:
- Speicheranforderungen: Speichern Nur Text: 15 GB/Tag -> = 5 TB/Jahr
Redundanz -> RF = 3 -> = 15 TB /Jahr -> 8 Festplatten a 2 TB -> 1 Unit
- CPU Anforderungen: 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) = 2000 Tweets/ Sekunde
Höchstleistung = 10 000 Tweets / Sekunde
Abdeckung von Systemausfällen und Anstieg der Nutzer = 20 000 Tweets/ Sekunden -> 40 Kerne mit 8 Threats (1.8 Ghz Base Clock)

Dimensionen Zielsystem kommerzielles System:
- CPU Anforderungen
Prototyp: 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) = 2000 Tweets/ Sekunde
Anforderung: Durchschnitt 6000 Tweets/ Sekunde
Höchster Messtand: 10 000 Tweets/ Sekunde
Abdeckung von Systemausällen = 20 000 Tweets/ Sekunden
Weiterentwicklung des Systems (Analyse von Videos, Bilder, andere Sprachen, etc.)
->x25 Prototypleistung Leistung
1000 Kerne mit 8 Threats (1.8 Ghz Base Clock)
- Speicher Anforderung
Twitter Daten Upload 12 TB/Tag
->4.5 PB/Jahr
45Unit x 8 Festplatten (2TB) = 720 TB
 -> 3 Racks a 45 Units für 1 Jahr


Fehler in der Analyse: 
-  Implementierung von Mechanismen zur Behandlung von Ausnahmen im Code, z. B. Try-Except-Blöcke
-  Implementierung eines Meldesystems und die manuelle Überprüfung der Fehler können die Falsch-Positiv sowie Falsch-Negativen Ereignisse behoben werden

Fehlertoleranz verteiltes System: 
- DataFrame wird zur Fehlertoleranz über mehrere Knoten in einem Cluster repliziert. Dies trägt zur Fehlertoleranz im Falle von Knotenausfällen bei. Die verlorenen Daten können von den Replikaten wiederhergestellt werden.
- Lineage Graph zur um verlorene Daten wiederherzustellen

Daten Speicherung:
- Twitter verwendet aktuell Hadoop-Cluster, auf denen Datenverarbeitung und HDFS laufen 
 + Einfache Anbindung, Synergien
 - Geschwindigkeit ausreichend für Realtime analyse?

Daten Codieung:
- Unstrukturierte Daten, wie der Twitter Feed und Text-Dateien, können nicht direkt im Parquet-Format gespeichert werden. Es gibt jedoch die Möglichkeit, unstrukturierte Daten zu kodieren oder zu transformieren, um sie in Parquet zu speichern.
- Daten werden in ein strukturiertes Format konvertieren (JSON), dieses wird dann in Parquet gespeichert 
- Prozess der Kodierung:
    1. Serialisierung: Die Twitter-Feed-Daten müssen in ein Format serialisiert werden, das auf die Festplatte geschrieben werden kann.
    2. Auf die Festplatte schreiben: Die serialisierten Twitter-Feed-Daten können unter Verwendung des Parquet-Dateiformats auf die Festplatte geschrieben werden.
    3. Von der Festplatte lesen: Die serialisierten Twitter-Feed-Daten können mit Hilfe der PySpark-API von der Festplatte zurückgelesen werden.
    4. Deserialisierung: Die Twitter-Feed-Daten können wieder in ihr ursprüngliches Format deserialisiert werden. Dies kann mit derselben Bibliothek geschehen, die für die Serialisierung verwendet wurde.

Abkürzungen des Prototyps:
- Limitierte Integration in Systemlandschaft
- Keine Analye von Videos, Bildern und Tonspuren
- Sentiment Analyse nur auf Englisch
- Tainingsdatensatz ist limietiert.

Echtzeitfähigkeit des Prototyps:
Durch 4 Kerne mit 8 Threats (1.8 Ghz Base Clock) führt zu einem Durchsatz von 2000 Tweets/Sekunde. Dies stellt bereits einen erheblichen Anteil dar, wenn man dies mit den durchschnittlichen Menge von 6000 Tweets/ Sekunde vergleicht. Jedoch handelt es sich hierbei um Tweets in reiner Textform. Für eine Live-Analyse, mit Metadaten sowie Tweets welche Bilder und Ton beinhalten,müsste das System ausgebaut werden. Dies könnte bspw. anhand von Spark Streaming mit einer horizontalen Skalierung durchgeführt werden.





## Spark Session

In [3]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import time
from pyspark.ml import PipelineModel

import os


MAX_MEMORY = "6g"
# spark = SparkSession.builder \
#                     .appName('live_hate_prediction')\
#                     .master("local[8]") \
#                     .config("spark.executor.memory", MAX_MEMORY) \
#                     .config("spark.driver.memory", MAX_MEMORY) \
#                     .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
#                     .getOrCreate()


spark = SparkSession \
    .builder \
    .appName("live_hate_prediction") \
    .master("local[8]") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print("Apache Spark version: ", spark.version)
spark

Apache Spark version:  3.3.1


## Streaming Source
Als Streaming-Quelle für den Prototyp nutzen wir das lokale Filesystem. Hier haben wir ca. 4 Mio. ungesehene Tweets, welche auf Hate-Nachrichten überprüft werden sollen. 
Dafür sind die Tweets mit Metadaten im Parquet-Format gespeichert (ca 5 GB Rohdaten).
Folgende Schritte sind hierfür notwendig:
- Einlesen des Daten-Schemas
- Erstellen eines ReadStreams auf den Daten (die maximale Anzahl der Files pro Trigger wird auf zwei begrenzt.)

In [3]:
spark_df = spark.read.parquet('C://data/twitter-stream-parquet-medium/')
dataSchema = spark_df.schema

print(f" Anzahl der Partitions: {spark_df.rdd.getNumPartitions()}")

 Anzahl der Partitions: 10


In [4]:
import pickle
hate_texts = spark_df.select('text').toPandas()['text']
hate_texts_list = hate_texts.to_list()
with open("../data/text_list", "wb") as fp:   #Pickling
    pickle.dump(hate_texts_list, fp)

In [5]:
with open("../data/text_list", "rb") as fp:   # Unpickling
        text_list = pickle.load(fp)

In [4]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 2).parquet('C://data/twitter-stream-parquet-medium/')

## Inferenz Pipeline laden
- Die zuvor erstelle Inferenzpipeline wird in den Hauptspeichergeladen.

In [5]:
trained_pipeline = PipelineModel.load("../models/mllib_model_nb")

## Batch Processing

Die Daten werden in Batches unterteilt und jeder Batch wird einzeln und parallel mit den Sparkoperationen verarbeitet. Die Ergebnisse der einzelnen Batches werden zum Endergebnis kombiniert. 

Folgende Methoden werden auf den Daten ausgeführt:
- Submethode clean_text wird auf jedem Micro-Batch ausgeführt und bereinigt die Texte (z.B. Groß- und Kleinschreibung, Sondernzeichen, ...)
- Methode batch_hate_interference wird auf jedem Micro-Batch ausgeführt und ruft Submethode clean_text auf, führt die Inferenz-Pipeline aus und speichert die identifizierten Hate-Tweets zur weiteren Verarbeitung. 

In [6]:
def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "(https?\://)\S+", "") # Remove links
  c = regexp_replace(c, "(\\n)|\n|\r|\t", "") # Remove CR, tab, and LR
  c = regexp_replace(c, "(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4})", "") # Remove dates
  c = regexp_replace(c, "@([A-Za-z0-9_]+)", "") # Remove usernames
  c = regexp_replace(c, "[0-9]", "") # Remove numbers
  c = regexp_replace(c, "\:|\/|\#|\.|\?|\!|\&|\"|\,", "") # Remove symbols
  return c

In [7]:
def batch_hate_inference(df, test):

    print(f'Iter: {test}')

    # Clean Text Columns
    df = df.withColumn("text", clean_text(col("text")))

    # Predict on Batch
    final_df = trained_pipeline.transform(df).select('text','prediction')

    # Handling for data (Save Hate Speech)
    hate = final_df.filter(final_df['prediction'] == 1.0)
    hate.write.mode('append').parquet('../data/parquet_output')
    times_hate = hate.count()
    print(f'Times hate detected: {times_hate}')

    not_hate = final_df.filter(final_df['prediction'] == 0.0)
    times_not_hate = not_hate.count()
    print(f'Times not hate detected: {times_not_hate}')


- Streaming-Methoden Foreach und ForeachBatch ermöglichen, dass eigene Logik auf den Stream ausgeführt werden kann. 
    - Trigger: availableNow ist nur für den Demo-Modus, beendet den Stream nachdem alle Verfügbaren Daten abgearbeitet wurden.
    - Start: Beginnt Ausführung 

In [8]:
batchHateInference = streaming.writeStream.foreachBatch(batch_hate_inference).trigger(availableNow=True).start() #

Iter: 0
Times hate detected: 234
Times not hate detected: 4857
Iter: 1
Times hate detected: 241
Times not hate detected: 4872
Iter: 2
Times hate detected: 251
Times not hate detected: 4853
Iter: 3
Times hate detected: 265
Times not hate detected: 4734
Iter: 4
Times hate detected: 223
Times not hate detected: 4590


Daten zu Diagnose und Protokoll der Ausführung:
- Mit dem aktuellen Setup (lokales Spark) können ca. 2000 Tweets / Sekunde untersucht werden.
- Durch Spark Streaming kann der Workload horizontal skaliert werden.

In [9]:
batchHateInference.recentProgress

[{'id': '2fa29151-9352-4ff4-af8c-2edb459a430e',
  'runId': 'e3e95956-d43d-4ac5-98ea-5dac46ff8553',
  'name': None,
  'timestamp': '2023-01-31T10:39:53.486Z',
  'batchId': 0,
  'numInputRows': 15273,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 1794.923022681866,
  'durationMs': {'addBatch': 7428,
   'getBatch': 102,
   'latestOffset': 275,
   'queryPlanning': 65,
   'triggerExecution': 8509,
   'walCommit': 282},
  'stateOperators': [],
  'sources': [{'description': 'FileStreamSource[file:/C:/data/twitter-stream-parquet-medium]',
    'startOffset': None,
    'endOffset': {'logOffset': 0},
    'latestOffset': None,
    'numInputRows': 15273,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 1794.923022681866}],
  'sink': {'description': 'ForeachBatchSink', 'numOutputRows': -1}},
 {'id': '2fa29151-9352-4ff4-af8c-2edb459a430e',
  'runId': 'e3e95956-d43d-4ac5-98ea-5dac46ff8553',
  'name': None,
  'timestamp': '2023-01-31T10:40:02.035Z',
  'batchId': 1,
  'numInputRow

In [10]:
batchHateInference.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

Folgende Tweets wurden durch das Modell als Hate-Speech identifiziert:

In [11]:
detected_hate_df = spark.read.parquet('../data/parquet_output')
hate_texts = detected_hate_df.select('text').toPandas()['text']
hate_texts.to_list()

['rt  giveaway 💸$ en usdt à gagner grâce à la dcf le launchpad qui révolutionne le financement du cinéma 🎥🎬🍿pour participer…',
 '    bryce is an ehhh kirsten was a miss tbh ',
 'rt  leeds cb cooper done his hamstring in the warmup against city dallas got a serious injury and stretched off firpo go…',
 'rt  🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿que mayo sea un buen mes🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿🧿',
 ' girl samez especially after today that thick veiny juicy schlong 🤤',
 'rt  fun fact these nike nft sneakers sold for  ethereum\xa0($ usd) ',
 'rt  do not spill stuff in his moms car ( of course i had to contribute to spidermanau and i couldn’t do it in a normal way…',
 'rt  socialist bussy is all inclusive ',
 '  some people just can’t take a joke xd',
 'rt  dos legados en una sola imagen comandante tomás borge martínez y el comandante walter ferretino hay legado sin hombres…',
 'goodnight ghoul and goodnight lash goodnight nobody goodnight hashs and goodnight to the gross lady whisperi

# Live Processing

In [14]:
KAFKA_TOPIC_NAME_CONS = "test-topic"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:29092'

print("Stream Data Processing Application Started ...")
print(time.strftime("%Y-%m-%d %H:%M:%S"))

# Construct a streaming DataFrame that reads from test-topic
live_tweets_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
    .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
    .option("startingOffsets", "latest") \
    .load()


def clean_text(c):
    c = lower(c)
    c = regexp_replace(c, "(https?\://)\S+", "") # Remove links
    c = regexp_replace(c, "(\\n)|\n|\r|\t", "") # Remove CR, tab, and LR
    c = regexp_replace(c, "(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4})", "") # Remove dates
    c = regexp_replace(c, "@([A-Za-z0-9_]+)", "") # Remove usernames
    c = regexp_replace(c, "[0-9]", "") # Remove numbers
    c = regexp_replace(c, "\:|\/|\#|\.|\?|\!|\&|\"|\,", "") # Remove symbols
    return c

trained_pipeline = PipelineModel.load("../models/mllib_model_nb")

# Define a schema for the orders data
tweets_schema = StructType() \
    .add("order_id", StringType()) \
    .add("text", StringType()) \
    .add("tweet_datetime", StringType())

print("Printing Schema of tweets_df: ")
live_tweets_df.printSchema()
tweets_df1 = live_tweets_df.selectExpr("CAST(value AS STRING)", "timestamp")

tweet_df2 = tweets_df1\
    .select(from_json(col("value"), tweets_schema).alias("data"), "timestamp").select("data.text", "timestamp")

# Clean Text Columns
tweet_df3 = tweet_df2.withColumn("text", clean_text(col("text")))

# Predict on Batch
final_df = trained_pipeline.transform(tweet_df3).select('text','prediction')

liveHateInference = final_df.writeStream \
    .format("console") \
    .trigger(continuous="1 second") \
    .start() \



liveHateInference.awaitTermination()

Stream Data Processing Application Started ...
2023-01-31 15:44:46
Printing Schema of tweets_df: 
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
liveHateInference.stop()

In [ ]:
liveHateInference.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}

In [9]:
liveHateInference.isActive

True

In [12]:
liveHateInference.recentProgress

[]